In [1]:
import Pkg
import DelimitedFiles
# Pkg.add("StatsKit")
# Pkg.add("random")
using StatsKit
using Random
using DelimitedFiles

In [28]:
struct mtDNA 
    r::Float64
    d::Float64
    m::Float64
    unique_id::Int
    parent_id::Int
    status::String
    
    function mtDNA(unique_id,parent_id,status)
        if status ∉ ["wild", "mutant"]
            error("Molecules must be of type 'wild' or 'mutant' ")
        end
        if status=="wild"
            new(0.01,0.01,0.001, unique_id, parent_id, status)
        elseif status=="mutant"
            new(0.01,0.01,0.0, unique_id, parent_id, status)
        end
    end
end 

In [3]:
rep(mol::mtDNA) = mol.r
deg(mol::mtDNA) = mol.d
mut(mol::mtDNA) = mol.m
unique_id(mol::mtDNA) = mol.unique_id
parent_id(mol::mtDNA) = mol.parent_id
status(mol::mtDNA) = mol.status 

status (generic function with 1 method)

In [4]:
function counter(system_state)::Vector{Int64}
    copy_num = length(system_state)
    W = sum([1 for x in system_state if status(x)=="wild"])
    return [W, copy_num-W]
end

counter (generic function with 1 method)

In [5]:
W0 = 150
M0 = 50
Nsim = 10000
Nout = 100
dt = Nsim/Nout;

In [46]:
function ABM_CE(Tmax::Int, dt::Int)
    N = trunc(Int, Tmax/dt)
    C0 = rand(Normal(200, 50), 1)
    h = rand(Beta(10,18.57), 1)
    W0 = round.( C0.*(1 .-h), digits=0)
    M0 = round.( C0.*h, digits=0)
    # initial state of system
    system_state = [mtDNA(x,-1,"wild") for x=1:W0 ]
    append!(system_state, [mtDNA(x,-1,"mutant") for x=W0+1:W0+M0] )
    current_id = trunc(Int, W0 + M0 + 1.0 )
    
    output = Matrix{Int}(undef, (N+1, 2))
    output[1,:] = counter(system_state)
    
    for i=1:Tmax
        if i%dt ==0
            output[trunc(Int, i/dt)+1,:] = counter(system_state)
        end
        
        molecules_to_remove = Vector{Int}()
        new_molecules = Vector{mtDNA}()
        
        for mol_ind=1:length(system_state)
        molecule = system_state[mol_ind]
        
        roll = rand(Float64)
        cdf = cumsum( [mut(molecule), deg(molecule), rep(molecule)] ) 
            if 0.0<roll && roll<cdf[1] # mutation
                append!(molecules_to_remove, mol_ind)
                for j=1:2
                    current_id += 1
                    daughter = mtDNA(current_id, unique_id(molecule), ["wild","mutant"][j])
                    push!(new_molecules, daughter)
                end
            elseif cdf[1]<roll && roll<cdf[2] # degredation
                append!( molecules_to_remove, mol_ind)
            
            elseif cdf[2]<roll && roll<cdf[3] # replication
                append!(molecules_to_remove, mol_ind)
                for j=1:2
                    current_id += 1
                    daughter = mtDNA(current_id, unique_id(molecule), status(molecule) )
                    push!(new_molecules, daughter)
                end
            end
        end
        system_state = [mol for (i,mol) in enumerate(system_state) if i ∉ molecules_to_remove ]
        append!(system_state, new_molecules)
    end
    return output
end

ABM_CE (generic function with 2 methods)

In [45]:
ABM_CE(1000,100)

LoadError: MethodError: no method matching (::Colon)(::Int64, ::Vector{Float64})
[0mClosest candidates are:
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real at range.jl:41
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Real} at range.jl:10
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T at range.jl:40
[0m  ...

In [29]:
system_state = [mtDNA(x,-1,"wild") for x=1:W0 ]
append!(system_state, [mtDNA(x,-1,"mutant") for x=W0+1:W0+M0] )
current_id = W0 + M0 + 1

output = Matrix{Int}(undef, (Nout+1, 2))
output[1,:] = counter(system_state)

for i = 1:Nsim
    if i%dt==0
        output[trunc(Int, i/dt)+1,:] = counter(system_state)
    end
    
    molecules_to_remove = Vector{Int}()
    new_molecules = Vector{mtDNA}()
    
    for mol_ind=1:length(system_state)
        molecule = system_state[mol_ind]
        
        roll = rand(Float64)
        cdf = cumsum( [mut(molecule), deg(molecule), rep(molecule)] ) 
        if 0.0<roll && roll<cdf[1] # mutation
            append!(molecules_to_remove, mol_ind)
            for j=1:2
                current_id += 1
                daughter = mtDNA(current_id, unique_id(molecule), ["wild","mutant"][j])
                push!(new_molecules, daughter)
            end
        elseif cdf[1]<roll && roll<cdf[2] # degredation
            append!( molecules_to_remove, mol_ind)
            
        elseif cdf[2]<roll && roll<cdf[3] # replication
            append!(molecules_to_remove, mol_ind)
            for j=1:2
                current_id += 1
                daughter = mtDNA(current_id, unique_id(molecule), status(molecule) )
                push!(new_molecules, daughter)
            end
        end
    end
    println(molecules_to_remove)
    println(new_molecules)
    system_state = [mol for (i,mol) in enumerate(system_state) if i ∉ molecules_to_remove ]
    append!(system_state, new_molecules)
end

[38, 97, 124, 151, 174, 197]
mtDNA[mtDNA(0.01, 0.01, 0.001, 202, 97, "wild"), mtDNA(0.01, 0.01, 0.001, 203, 97, "wild"), mtDNA(0.01, 0.01, 0.0, 204, 151, "mutant"), mtDNA(0.01, 0.01, 0.0, 205, 151, "mutant"), mtDNA(0.01, 0.01, 0.0, 206, 174, "mutant"), mtDNA(0.01, 0.01, 0.0, 207, 174, "mutant")]
[168, 199]
mtDNA[]
[17, 18, 19, 77, 115, 148]
mtDNA[mtDNA(0.01, 0.01, 0.001, 208, 18, "wild"), mtDNA(0.01, 0.01, 0.001, 209, 18, "wild"), mtDNA(0.01, 0.01, 0.001, 210, 19, "wild"), mtDNA(0.01, 0.01, 0.001, 211, 19, "wild"), mtDNA(0.01, 0.01, 0.001, 212, 117, "wild"), mtDNA(0.01, 0.01, 0.001, 213, 117, "wild")]
[51, 57, 79, 82, 106]
mtDNA[mtDNA(0.01, 0.01, 0.001, 214, 55, "wild"), mtDNA(0.01, 0.01, 0.001, 215, 55, "wild"), mtDNA(0.01, 0.01, 0.001, 216, 61, "wild"), mtDNA(0.01, 0.01, 0.001, 217, 61, "wild"), mtDNA(0.01, 0.01, 0.001, 218, 87, "wild"), mtDNA(0.01, 0.01, 0.001, 219, 87, "wild"), mtDNA(0.01, 0.01, 0.001, 220, 112, "wild"), mtDNA(0.01, 0.01, 0.001, 221, 112, "wild")]
[12, 57, 66, 68, 

[9, 71, 109]
mtDNA[mtDNA(0.01, 0.01, 0.001, 414, 31, "wild"), mtDNA(0.01, 0.01, 0.001, 415, 31, "wild"), mtDNA(0.01, 0.01, 0.0, 416, 310, "mutant"), mtDNA(0.01, 0.01, 0.0, 417, 310, "mutant")]
[10, 71, 138, 148, 149, 172]
mtDNA[mtDNA(0.01, 0.01, 0.001, 418, 33, "wild"), mtDNA(0.01, 0.01, 0.001, 419, 33, "wild"), mtDNA(0.01, 0.01, 0.001, 420, 242, "wild"), mtDNA(0.01, 0.01, 0.001, 421, 242, "wild"), mtDNA(0.01, 0.01, 0.0, 422, 382, "mutant"), mtDNA(0.01, 0.01, 0.0, 423, 382, "mutant"), mtDNA(0.01, 0.01, 0.0, 424, 407, "mutant"), mtDNA(0.01, 0.01, 0.0, 425, 407, "mutant")]
[3, 66, 124, 149, 158, 179]
mtDNA[mtDNA(0.01, 0.01, 0.001, 426, 227, "wild"), mtDNA(0.01, 0.01, 0.0, 427, 227, "mutant"), mtDNA(0.01, 0.01, 0.001, 428, 352, "wild"), mtDNA(0.01, 0.01, 0.001, 429, 352, "wild"), mtDNA(0.01, 0.01, 0.001, 430, 420, "wild"), mtDNA(0.01, 0.01, 0.001, 431, 420, "wild")]
[29, 42, 90, 158, 180]
mtDNA[mtDNA(0.01, 0.01, 0.001, 432, 83, "wild"), mtDNA(0.01, 0.01, 0.0, 433, 83, "mutant")]
[66, 169]

mtDNA[mtDNA(0.01, 0.01, 0.0, 728, 169, "mutant"), mtDNA(0.01, 0.01, 0.0, 729, 169, "mutant"), mtDNA(0.01, 0.01, 0.001, 730, 209, "wild"), mtDNA(0.01, 0.01, 0.001, 731, 209, "wild"), mtDNA(0.01, 0.01, 0.0, 732, 296, "mutant"), mtDNA(0.01, 0.01, 0.0, 733, 296, "mutant"), mtDNA(0.01, 0.01, 0.0, 734, 358, "mutant"), mtDNA(0.01, 0.01, 0.0, 735, 358, "mutant"), mtDNA(0.01, 0.01, 0.001, 736, 596, "wild"), mtDNA(0.01, 0.01, 0.001, 737, 596, "wild"), mtDNA(0.01, 0.01, 0.001, 738, 654, "wild"), mtDNA(0.01, 0.01, 0.001, 739, 654, "wild")]
[74, 85, 130]
mtDNA[mtDNA(0.01, 0.01, 0.001, 740, 556, "wild"), mtDNA(0.01, 0.01, 0.0, 741, 556, "mutant")]
[14, 130, 215]
mtDNA[mtDNA(0.01, 0.01, 0.001, 742, 248, "wild"), mtDNA(0.01, 0.01, 0.001, 743, 248, "wild"), mtDNA(0.01, 0.01, 0.0, 744, 741, "mutant"), mtDNA(0.01, 0.01, 0.0, 745, 741, "mutant")]
[16, 30, 31, 63, 135, 169, 174, 212]
mtDNA[mtDNA(0.01, 0.01, 0.001, 746, 274, "wild"), mtDNA(0.01, 0.01, 0.001, 747, 274, "wild"), mtDNA(0.01, 0.01, 0.0, 748, 39

mtDNA[mtDNA(0.01, 0.01, 0.001, 892, 472, "wild"), mtDNA(0.01, 0.01, 0.001, 893, 472, "wild"), mtDNA(0.01, 0.01, 0.001, 894, 566, "wild"), mtDNA(0.01, 0.01, 0.001, 895, 566, "wild"), mtDNA(0.01, 0.01, 0.001, 896, 859, "wild"), mtDNA(0.01, 0.01, 0.001, 897, 859, "wild"), mtDNA(0.01, 0.01, 0.001, 898, 871, "wild"), mtDNA(0.01, 0.01, 0.001, 899, 871, "wild")]
[27, 45, 97, 143, 158]
mtDNA[mtDNA(0.01, 0.01, 0.0, 900, 711, "mutant"), mtDNA(0.01, 0.01, 0.0, 901, 711, "mutant"), mtDNA(0.01, 0.01, 0.001, 902, 803, "wild"), mtDNA(0.01, 0.01, 0.001, 903, 803, "wild")]
[6, 107]
mtDNA[]
[90, 102]
mtDNA[mtDNA(0.01, 0.01, 0.0, 904, 703, "mutant"), mtDNA(0.01, 0.01, 0.0, 905, 703, "mutant"), mtDNA(0.01, 0.01, 0.001, 906, 727, "wild"), mtDNA(0.01, 0.01, 0.001, 907, 727, "wild")]
[2, 9, 48, 181]
mtDNA[mtDNA(0.01, 0.01, 0.001, 908, 303, "wild"), mtDNA(0.01, 0.01, 0.0, 909, 303, "mutant"), mtDNA(0.01, 0.01, 0.001, 910, 590, "wild"), mtDNA(0.01, 0.01, 0.001, 911, 590, "wild")]
[70, 183]
mtDNA[mtDNA(0.01, 0.

mtDNA[mtDNA(0.01, 0.01, 0.0, 1206, 864, "mutant"), mtDNA(0.01, 0.01, 0.0, 1207, 864, "mutant"), mtDNA(0.01, 0.01, 0.001, 1208, 1147, "wild"), mtDNA(0.01, 0.01, 0.001, 1209, 1147, "wild")]
[88, 104, 166, 168]
mtDNA[mtDNA(0.01, 0.01, 0.001, 1210, 1021, "wild"), mtDNA(0.01, 0.01, 0.0, 1211, 1021, "mutant"), mtDNA(0.01, 0.01, 0.0, 1212, 1056, "mutant"), mtDNA(0.01, 0.01, 0.0, 1213, 1056, "mutant"), mtDNA(0.01, 0.01, 0.001, 1214, 1149, "wild"), mtDNA(0.01, 0.01, 0.001, 1215, 1149, "wild")]
[33, 60, 65, 71, 129, 151, 182]
mtDNA[mtDNA(0.01, 0.01, 0.001, 1216, 960, "wild"), mtDNA(0.01, 0.01, 0.001, 1217, 960, "wild"), mtDNA(0.01, 0.01, 0.0, 1218, 1101, "mutant"), mtDNA(0.01, 0.01, 0.0, 1219, 1101, "mutant"), mtDNA(0.01, 0.01, 0.0, 1220, 1129, "mutant"), mtDNA(0.01, 0.01, 0.0, 1221, 1129, "mutant"), mtDNA(0.01, 0.01, 0.001, 1222, 1168, "wild"), mtDNA(0.01, 0.01, 0.001, 1223, 1168, "wild")]
[21, 141, 176, 193]
mtDNA[mtDNA(0.01, 0.01, 0.0, 1224, 729, "mutant"), mtDNA(0.01, 0.01, 0.0, 1225, 729, "

...

mtDNA[mtDNA(0.01, 0.01, 0.001, 9676, 9302, "wild"), mtDNA(0.01, 0.01, 0.001, 9677, 9302, "wild"), mtDNA(0.01, 0.01, 0.0, 9678, 9465, "mutant"), mtDNA(0.01, 0.01, 0.0, 9679, 9465, "mutant"), mtDNA(0.01, 0.01, 0.0, 9680, 9466, "mutant"), mtDNA(0.01, 0.01, 0.0, 9681, 9466, "mutant"), mtDNA(0.01, 0.01, 0.0, 9682, 9511, "mutant"), mtDNA(0.01, 0.01, 0.0, 9683, 9511, "mutant"), mtDNA(0.01, 0.01, 0.0, 9684, 9532, "mutant"), mtDNA(0.01, 0.01, 0.0, 9685, 9532, "mutant"), mtDNA(0.01, 0.01, 0.0, 9686, 9608, "mutant"), mtDNA(0.01, 0.01, 0.0, 9687, 9608, "mutant"), mtDNA(0.01, 0.01, 0.0, 9688, 9624, "mutant"), mtDNA(0.01, 0.01, 0.0, 9689, 9624, "mutant"), mtDNA(0.01, 0.01, 0.0, 9690, 9633, "mutant"), mtDNA(0.01, 0.01, 0.0, 9691, 9633, "mutant")]
[66, 74, 88, 229, 235, 296, 309, 386]
mtDNA[mtDNA(0.01, 0.01, 0.001, 9692, 8907, "wild"), mtDNA(0.01, 0.01, 0.001, 9693, 8907, "wild"), mtDNA(0.01, 0.01, 0.001, 9694, 9440, "wild"), mtDNA(0.01, 0.01, 0.001, 9695, 9440, "wild"), mtDNA(0.01, 0.01, 0.0, 9696, 9

[52, 105, 204, 275, 298, 334, 366, 378, 409, 432]
mtDNA[mtDNA(0.01, 0.01, 0.0, 9842, 8874, "mutant"), mtDNA(0.01, 0.01, 0.0, 9843, 8874, "mutant"), mtDNA(0.01, 0.01, 0.0, 9844, 9234, "mutant"), mtDNA(0.01, 0.01, 0.0, 9845, 9234, "mutant"), mtDNA(0.01, 0.01, 0.0, 9846, 9717, "mutant"), mtDNA(0.01, 0.01, 0.0, 9847, 9717, "mutant"), mtDNA(0.01, 0.01, 0.0, 9848, 9755, "mutant"), mtDNA(0.01, 0.01, 0.0, 9849, 9755, "mutant"), mtDNA(0.01, 0.01, 0.001, 9850, 9771, "wild"), mtDNA(0.01, 0.01, 0.001, 9851, 9771, "wild"), mtDNA(0.01, 0.01, 0.0, 9852, 9830, "mutant"), mtDNA(0.01, 0.01, 0.0, 9853, 9830, "mutant")]
[83, 84, 89, 97, 109, 139, 172, 198, 233, 305, 311, 405, 424]
mtDNA[mtDNA(0.01, 0.01, 0.0, 9854, 9134, "mutant"), mtDNA(0.01, 0.01, 0.0, 9855, 9134, "mutant"), mtDNA(0.01, 0.01, 0.0, 9856, 9152, "mutant"), mtDNA(0.01, 0.01, 0.0, 9857, 9152, "mutant"), mtDNA(0.01, 0.01, 0.0, 9858, 9263, "mutant"), mtDNA(0.01, 0.01, 0.0, 9859, 9263, "mutant"), mtDNA(0.01, 0.01, 0.0, 9860, 9461, "mutant"), mt

mtDNA[mtDNA(0.01, 0.01, 0.0, 10148, 8590, "mutant"), mtDNA(0.01, 0.01, 0.0, 10149, 8590, "mutant"), mtDNA(0.01, 0.01, 0.001, 10150, 8897, "wild"), mtDNA(0.01, 0.01, 0.001, 10151, 8897, "wild"), mtDNA(0.01, 0.01, 0.001, 10152, 9155, "wild"), mtDNA(0.01, 0.01, 0.0, 10153, 9155, "mutant"), mtDNA(0.01, 0.01, 0.0, 10154, 9264, "mutant"), mtDNA(0.01, 0.01, 0.0, 10155, 9264, "mutant"), mtDNA(0.01, 0.01, 0.0, 10156, 9622, "mutant"), mtDNA(0.01, 0.01, 0.0, 10157, 9622, "mutant"), mtDNA(0.01, 0.01, 0.0, 10158, 9628, "mutant"), mtDNA(0.01, 0.01, 0.0, 10159, 9628, "mutant"), mtDNA(0.01, 0.01, 0.0, 10160, 10057, "mutant"), mtDNA(0.01, 0.01, 0.0, 10161, 10057, "mutant")]
[67, 115, 203, 204, 244, 248, 301, 389, 442]
mtDNA[mtDNA(0.01, 0.01, 0.001, 10162, 9803, "wild"), mtDNA(0.01, 0.01, 0.001, 10163, 9803, "wild"), mtDNA(0.01, 0.01, 0.0, 10164, 9884, "mutant"), mtDNA(0.01, 0.01, 0.0, 10165, 9884, "mutant"), mtDNA(0.01, 0.01, 0.0, 10166, 9890, "mutant"), mtDNA(0.01, 0.01, 0.0, 10167, 9890, "mutant"), m

Excessive output truncated after 524288 bytes.

wild"), mtDNA(0.01, 0.01, 0.001, 10331, 9777, "wild"), mtDNA(0.01, 0.01, 0.0, 10332, 10136, "mutant"), mtDNA(0.01, 0.01, 0.0, 10333, 10136, "mutant"), mtDNA(0.01, 0.01, 0.001, 10334, 10152, "wild"), mtDNA(0.01, 0.01, 0.001, 10335, 10152, "wild"), mtDNA(0.01, 0.01, 0.0, 10336, 10294, "mutant"), mtDNA(0.01, 0.01, 0.0, 10337, 10294, "mutant")]
[194, 199, 279, 287, 358, 359, 370, 454]
mtDNA[mtDNA(0.01, 0.01, 0.0, 10338, 9944, "mutant"), mtDNA(0.01, 0.01, 0.0, 10339, 9944, "mutant"), mtDNA(0.01, 0.01, 0.001, 10340, 10117, "wild"), mtDNA(0.01, 0.01, 0.001, 10341, 10117, "wild")]
[11, 44, 65, 145, 153, 169, 213, 215, 257, 262, 271, 404, 434, 456]
mtDNA[mtDNA(0.01, 0.01, 0.0, 10342, 9284, "mutant"), mtDNA(0.01, 0.01, 0.0, 10343, 9284, "mutant"), mtDNA(0.01, 0.01, 0.001, 10344, 9429, "wild"), mtDNA(0.01, 0.01, 0.001, 10345, 9429, "wild"), mtDNA(0.01, 0.01, 0.0, 10346, 9799, "mutant"), mtDNA(0.01, 0.01, 0.0, 10347, 9799, "mutant"), mtDNA(0.01, 0.01, 0.0, 10348, 10065, "mutant"), mtDNA(0.01, 0.01

In [9]:
writedlm("ABMsim.txt", output)

In [40]:
tt = Vector{Int}()
append!(tt, [1 2])
append!(tt, [3 4])
tt

4-element Vector{Int64}:
 1
 2
 3
 4